# Data Visualization and Exploration

Now that we've trudged through getting our data into a format that we can work with, it's time to *explore the data*. The process of data examination can be done in 2 ways:

1. **Hypothesis-driven examination**

Hypothesis-driven exploration is the process of probing the data for specific phenomena or behaviours. In this case we have very specific ideas of what we're interested in and our visualization process will reflect this. Take for example a hypothesis about changing reaction time distributions under different drugs (alcohol, caffeine, etc...). We might specifically examine the reaction time distributions across different drugs that were administered during a study. 

In some really specific cases one might attempt to simulate a phenomena using a model that might produce an output of many functions (reaction time, accuracy etc...), then perform the same computation on the data and compare the model to what is observed.


2. **Data-driven exploration**

With an increasing amount of available data, data-driven exploration is becoming more popular. In this case we have a data sample with loose ideas about what we'd like to extract from it. To get a better idea of what the data may be telling us we might perform different types of visualization and computation in order to extract meaningful information from the data. One key issue with data-driven exploration is that it is easily subject to statistical fallacies (multiple testing errors etc..). Visualization of data with the addition with careful statistical testing can result in meaningful findings that are generalizable across multiple datasets.

***

We're going to focus on data-driven exploration where we visualize the relationships between our observed variables and try to draw inference about a phenomena which may be apparent in our data. Let's pretend that our goal is to understand what components of our data drive differences in their cognitive performance. These effects could stem from:


1. Gender differences in performance
2. Group (case/control) differences in performance
3. The effect of brain volume/surface area/thickness on performance
4. Some complex interaction effects where the effect of a variable B on performance is dependent on another variable A. 


We'll get a high-level overview of how visualization can help us explore these phenomena within our dataset. Note that we will not get into statistical analysis. Although Python *does* have statistical libraries, I would heavily recommend that one gets comfortable using it for data management, pre-processing, visualization, and automation. Actual statistical analysis is best done in languages like *R* which has a more sophisticated set of statistical methods compared to Python (although Python is slowly catching up!)

In [ ]:
df.info()

In this module we'll be using a fantastic package called <code>seaborn</code> which is a statistical visualization library made in Python. If you're familiar with **R**, this tries to replicate some of the functionality of **ggplot**. The nice thing about **Python** is that algorithms for pre-processing data are easier to apply and visualize compared to a language like **R** where generally you don't write up algorithms in the first place (it's slow and R's syntax makes it tedious)

Let's start easy. Suppose we're interested in the effect of gender on cognitive scores. Seaborn works directly with <code>DataFrame</code> objects to generate visualizations in a way that is easy to do! All we need to do is to specify which columns are the x axis and which are the y axis:

We could also get a bit more sophisicated. Suppose we wanted to also account for which group a person is in:

As you can see there is no gender effect on cognitive performance, but there is a pretty clear group effect! Notice how <code>seaborn</code> creates legends for you!

Now boxplots are nice but they hide a lot of the detail that might be useful for us in terms of understanding the distribution of the data. We can use <code>sns.boxenplot</code> to get an idea of the quantiles of the data or <code>sns.violinplot</code> to view a smoothed distribution of the data:

In [ ]:
sns.boxenplot(x='gender',y='cog_score',data=df,hue='group')

Now let's look into some more sophisticated factors. What about the effect of brain measurements?

In [ ]:
sns.scatterplot(x='brain_surfarea',y='cog_score',data=df,hue='group')

Looks like brain surface area and brain grey matter thickness do not clearly relate to cognitve scoring. Interestingly *brain grey matter thickness* seems to be a decent indicator!

We could just try plotting every variable one by one and looking for relationships between the data and this could take *forever* if there are a ton of variables. Hypothesis-driven exploration is important for this reason: we can narrow our variables of interest down significantly to a manageable level!

With our case we don't have too many variables, so its possible to simply just plot every variable against each other:

This is achieved by using <code>sns.pairplot</code> which plots every variable against each other:

And we get an error. This error says that our DataFrame cannot contain any values called "infs" or "NaNs". We definitely know that our data contains "NaNs" because we put them there. But using NaNs actually works to our advantage because pandas contains utilities to deal with "NaNs" specifically;

Looking at the cognitive score row it is pretty apparent that there seems to be an interaction between cognitive score, group, and some of the measured variables. We can perform the same visualization for gender as well:

The separation is less clear here, we do see slight differences in brain volume but not much else. What about if we wanted to separate out by:

- male control
- female control
- male case
- female case

Turns out that this is slightly complicated. Seaborn can only use one column to specify as a "grouping" variable. Therefore, in order to combine these groups we need to merge these 2 columns (2 distinctions each) into a single column. One way to do this is to simply concatenate the strings for each column together:

Looks like a large degree of separation exists against case and control and the only axis in which male and female differ is in brain volumes. This makes sense with our intuition, we expect that males and females will perform equally on the cognitive task and it is true that male and female brain volumes differ slightly. Furthermore the only mean shift in cognitive performance seems to be related to whether an individual is a case or control. 

Furthermore, it indicates that if we want to standardize data, we'll need to account for the fact that we have different means between our groups. It's entirely possible that additional sub-groups exist but the current description of our data using group and genders seems to account for all mean shifts in our data enough to be useful. 

For the final analysis phase, we'll explore how correlations matrices can be computed from the data in order to examine not only group mean shifts in measurements but how variables may relate to each-other

***

## Computing Correlations

The examples above explore the general group-wise **covariance structure** of the data. In other words, how do variables change with respect to other each other. If we wanted to assess the strength of this relationship statistically then we might want to compute correlations between each element of the data to generate a full correlation matrix. 

Calculating a correlation matrix is incredibly simple once the data is in the correct format. We first need to standardize our data so that our correlations between variables of interest make sense. To do this we'll first standardize our data group-wise and with the assumption of equal variance and no gender interaction effect, we can compute differences in correlation between control and case participants. 

The first step is to compute within-class Z-scores so that all data is centered around 0 and has unit variance. This is done by looping over each class, then updating the dataframe with Z-scored values:

It is *almost always* recommended that when you compute statistical estimates such as correlation that you verify what you're calculating through visualization. If we just went ahead and computed a correlation matrix without first visualizing our data our correlation matrix would be spurious and useless for further analysis.

Computing a correlation matrix is incredibly easy and is done with <code>DataFrame.corr</code>. First we'll divide out a case and control dataframe: